<a href="https://colab.research.google.com/github/Hongbi-Kim/Writing_lyrics/blob/master/KoGPT2_%EA%B0%80%EC%82%AC%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 57.6 MB/s 
     |████████████████████████████████| 77 kB 9.2 MB/s 
     |████████████████████████████████| 6.5 MB 57.8 MB/s 
     |████████████████████████████████| 596 kB 49.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
# 트랜스포머 라이브러리로 사전 학습된 한국어 GPT-2 모델 사용
import torch
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast

In [2]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [3]:
# KoGPT2 Tokenizer 다루기
# 허깅페이스의 토크나이저 라이브러리를 사용해 입력 텍스트의 토큰 ID 값을 얻을 수 있다.
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [4]:
# 문장생성하기
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


# 2. 문장 생성 모델 파인튜닝

In [14]:
!pip install ratsnlp # 의존성 패키지 설치

     |████████████████████████████████| 42 kB 914 kB/s 
     |████████████████████████████████| 806 kB 9.0 MB/s 
     |████████████████████████████████| 57 kB 6.7 MB/s 
     |████████████████████████████████| 2.8 MB 89.9 MB/s 
     |████████████████████████████████| 136 kB 99.4 MB/s 
     |████████████████████████████████| 636 kB 87.2 MB/s 
     |████████████████████████████████| 829 kB 71.1 MB/s 
     |████████████████████████████████| 398 kB 90.1 MB/s 
     |████████████████████████████████| 3.3 MB 79.3 MB/s 
     |████████████████████████████████| 1.1 MB 81.7 MB/s 
     |████████████████████████████████| 96 kB 7.4 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
     |████████████████████████████████| 144 kB 99.9 MB/s 
     |████████████████████████████████| 271 kB 101.1 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=5c43afd66d593c2d695bb16cc63ed751bd584297b114da4c14293ddbaa0a40a8
  Stored in directory: /root/.cache/pip/whee

In [16]:
# 모델 환경 설정
# KoGPT2 모델을 nsmc으로 파인튜닝
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
args = GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2", # 프리트레인 마친 언어 모델의 이름(단, 해당 모델은 허깅펭이스 모델 허브에 등록되어야 함.)
    downstream_corpus_name="nsmc", # 다운스트림 데이터 이름
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation", # 다운스트림 데이터를 내려받을 위치. 
    max_seq_length=32, # 토큰 기준 입력 문장 최대 길이. 이보다 긴 문장은 max_seq_length로 자르고, 짧은 문장은 max_seq_length가 되도록 스페셜 토큰 ([PAD])를 붙여줌
    batch_size=32 if torch.cuda.is_available() else 4, # 배치 크기. 하드웨어 가속기로 GPU 선택 시 (True) 32, TPU라면 4.
    learning_rate=5e-5,  # 러닝 레이트(보폭). 1회 스텝에서 모델을 얼마나 업데이트할지에 관한 크기를 가맂킴.
    epochs=3, # 학습 에포크 수, 3이라면 학습 데이터 전체를 3회 반복 학습
    tpu_cores=0 if torch.cuda.is_available() else 8, # TPU 코어 수. 하드웨어 가속기로 GPU를 선택했다면 0, TPU라면 8
    seed=7,
)

In [ ]:
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
args = GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2", # 프리트레인 마친 언어 모델의 이름(단, 해당 모델은 허깅펭이스 모델 허브에 등록되어야 함.)
    downstream_corpus_name="nsmc", # 다운스트림 데이터 이름
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation", # 다운스트림 데이터를 내려받을 위치. 
    max_seq_length=32, # 토큰 기준 입력 문장 최대 길이. 이보다 긴 문장은 max_seq_length로 자르고, 짧은 문장은 max_seq_length가 되도록 스페셜 토큰 ([PAD])를 붙여줌
    batch_size=32 if torch.cuda.is_available() else 4, # 배치 크기. 하드웨어 가속기로 GPU 선택 시 (True) 32, TPU라면 4.
    learning_rate=5e-5,  # 러닝 레이트(보폭). 1회 스텝에서 모델을 얼마나 업데이트할지에 관한 크기를 가맂킴.
    epochs=3, # 학습 에포크 수, 3이라면 학습 데이터 전체를 3회 반복 학습
    tpu_cores=0 if torch.cuda.is_available() else 8, # TPU 코어 수. 하드웨어 가속기로 GPU를 선택했다면 0, TPU라면 8
    seed=7,
)

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_pickle('/content/drive/MyDrive/5조_파이널PJT/코드/data/melon_mecab.pkl')

- 랜덤 시드 고정\
학습 재현을 위해 랜덤 시드를 고정합니다.

In [17]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


- 로거 설정\
  메세지 출력 등을 위한 logger를 설정합니다.

In [22]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-generation', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=8, fp16=False, tpu_cores=0)


- 말뭉치 다운로드\
실습에 사용할 말뭉치(NSMC)를 다운로드합니다.

In [23]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=args.force_download,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 100MB/s]                             
[nsmc] download ratings_test.txt: 4.90MB [00:00, 42.6MB/s]                            


In [5]:
df1 = df.reset_index()

In [20]:
ballad = df1.iloc[:71041]
dance = df1.iloc[71041:]

In [41]:
dance['genres_id'] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
dance = dance['lyric']
dance

71041    Red Velvet\n\n무도회를 뒤집어\n작은 소란을 또 일으켜\nThis is ...
71042    Falling in love\n너에겐 난 Option\n시작부터 다른 너와 나\n깨...
71043    RUN\nTold you not 또 괜한 기대\n겉으론 걱정해 난 안 바뀌네\n또 ...
71044    울지 마 울지 마\n어린아이같이\n웃는 게 웃는 게\n이기는 거라고\nYou are...
71045    따분한 나의 눈빛이\n무표정했던 얼굴이\n널 보며 빛나고 있어\n널 담은 눈동자는 ...
                               ...                        
94608    하지마 하지마 음주운전 하지마\n하지마 하지마 음주운전 하지마\n술먹고 운전했었어\...
94609    Merry christmas\n큰마음 먹고 널 위해 준비한 데이트\n나 홀로 집에 ...
94610    저 떨어지는 별에 빌었어\n지금 네 맘도 나와 똑같기를\n잠든 밤이 지나고 네가 떠...
94611    소중한 무언가를 난 또 쫓고 있어\n어떤 모양인지조차\n생각해 볼 순간도 없고\n너...
94612    마시자 잔을 들어 친구야\n사느라고 고생이 많다\n막힌 현실에 갇혀 살아도\n잘 버...
Name: lyric, Length: 23572, dtype: object

In [17]:
dance.to_csv('/content/drive/MyDrive/Colab Notebooks/writing_lyrics/dance_lyric.txt')

In [29]:
#file = open'/content/drive/MyDrive/Colab Notebooks/writing_lyrics/dance_lyric.txt', 'w')

import os
def write_lines(path, lines):
    with open(path, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(f'{line},')
write_lines('/content/drive/MyDrive/Colab Notebooks/writing_lyrics/dance_lyric.txt',dance.values )

In [30]:
!head '/content/drive/MyDrive/Colab Notebooks/writing_lyrics/dance_lyric.txt'

Red Velvet

무도회를 뒤집어
작은 소란을 또 일으켜
This is gonna be a crazy night
(To-na-na-na-na-na-night)

등장부터 인사까지 파격적이게
우리 살짝 놀아볼까
솔직하게 Bae bae


In [31]:
!tail '/content/drive/MyDrive/Colab Notebooks/writing_lyrics/dance_lyric.txt'

다 잊고 잔을 들어 친구야
건배하자 내일을 위해
장담하는데 오늘보다는
더 나아질 거야
힘내라 멋진 나의 친구야
너는 이미 행복한 거야
함께 슬퍼해줄 친구가 있다
웃어라 친구야
함께 슬퍼해줄 친구가 있다
웃어라 친구야,

In [35]:
import os
os.makedirs("/gdrive/My Drive/nlpbook/bbpe", exist_ok=True)

In [39]:
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=['/content/drive/MyDrive/Colab Notebooks/writing_lyrics/dance_lyric.txt'],
    vocab_size=100000,
    special_tokens=["[PAD]"]
)
bytebpe_tokenizer.save_model("/gdrive/My Drive/nlpbook/bbpe")

['/gdrive/My Drive/nlpbook/bbpe/vocab.json',
 '/gdrive/My Drive/nlpbook/bbpe/merges.txt']

In [40]:
!cat /gdrive/My\ Drive/nlpbook/bbpe/vocab.json

{"[PAD]":0,"!":1,"\"":2,"#":3,"$":4,"%":5,"&":6,"'":7,"(":8,")":9,"*":10,"+":11,",":12,"-":13,".":14,"/":15,"0":16,"1":17,"2":18,"3":19,"4":20,"5":21,"6":22,"7":23,"8":24,"9":25,":":26,";":27,"<":28,"=":29,">":30,"?":31,"@":32,"A":33,"B":34,"C":35,"D":36,"E":37,"F":38,"G":39,"H":40,"I":41,"J":42,"K":43,"L":44,"M":45,"N":46,"O":47,"P":48,"Q":49,"R":50,"S":51,"T":52,"U":53,"V":54,"W":55,"X":56,"Y":57,"Z":58,"[":59,"\\":60,"]":61,"^":62,"_":63,"`":64,"a":65,"b":66,"c":67,"d":68,"e":69,"f":70,"g":71,"h":72,"i":73,"j":74,"k":75,"l":76,"m":77,"n":78,"o":79,"p":80,"q":81,"r":82,"s":83,"t":84,"u":85,"v":86,"w":87,"x":88,"y":89,"z":90,"{":91,"|":92,"}":93,"~":94,"¡":95,"¢":96,"£":97,"¤":98,"¥":99,"¦":100,"§":101,"¨":102,"©":103,"ª":104,"«":105,"¬":106,"®":107,"¯":108,"°":109,"±":110,"²":111,"³":112,"´":113,"µ":114,"¶":115,"·":116,"¸":117,"¹":118,"º":119,"»":120,"¼":121,"½":122,"¾":123,"¿":124,"À":125,"Á":126,"Â":127,"Ã":128,"Ä":129,"Å":130,"Æ":131,"Ç":132,"È":133,"É":134,"Ê":135,"Ë":136,"Ì":137

In [41]:
!head /gdrive/My\ Drive/nlpbook/bbpe/merges.txt

#version: 0.2 - Trained by `huggingface/tokenizers`
Ġ ë
Ġ ì
ì Ŀ
í ķ
Ġ ê
ë Ĥ
ì ķ
ìĿ ´
ì §


# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다

## GPT 입력값 만들기

GPT 모델 입력값을 만들려면 Byte-level Byte Pair Encoding 어휘집합 구축 결과(`vocab.json`, `merges.txt`)가 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 있어야 합니다. 다음을 수행해 이미 만들어 놓은 BBPE 어휘집합을 포함한 GPT 토크나이저를 `tokenizer_gpt`라는 변수로 선언합니다.

In [43]:
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/gdrive/My Drive/nlpbook/bbpe")
tokenizer_gpt.pad_token = "[PAD]"

예시 문장 세 개를 각각 토큰화해보겠습니다.

In [47]:
df.lyric.head()

0    너와 함께 하고 싶은 일들을\n상상하는 게\n요즘 내 일상이 되고\n너의 즐거워하는...
1    뭐하고 있었니 늦었지만\n잠시 나올래\n너의 집 골목에 있는\n놀이터에 앉아 있어\...
2    우리 함께 듣던 그 노랠 듣고 싶을까\n듣고 나서 잠시 날 생각은 할까\n아주 혹시...
3    눈물이 난다 이 길을 걸으면\n그 사람 손길이 자꾸 생각이 난다\n붙잡지 못하고 가...
4    둘이 손잡고 걸을 때\n마주 보며 또 웃을 때\n사랑한다 말하지 않아도 알아\n널 ...
Name: lyric, dtype: object

In [56]:
sentences = [df.lyric[0],df.lyric[1],df.lyric[2]]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인합니다.

In [57]:
tokenized_sentences

[['ëĦĪìĻĢ',
  'Ġíķ¨ê»ĺ',
  'Ġíķĺê³ł',
  'Ġìĭ¶ìĿĢ',
  'ĠìĿ¼ëĵ¤ìĿĦ',
  'Ċ',
  'ìĥģìĥģíķĺëĬĶ',
  'Ġê²Į',
  'Ċ',
  'ìļĶì¦ĺ',
  'ĠëĤ´',
  'ĠìĿ¼ìĥģìĿ´',
  'ĠëĲĺê³ł',
  'Ċ',
  'ëĦĪìĿĺ',
  'Ġì¦Ĳê±°ìĽĮ',
  'íķĺëĬĶ',
  'Ġëª¨ìĬµìĿĦ',
  'Ġë³´ê³ł',
  'ĠìŀĪìľ¼ë©´',
  'Ċ',
  'ìŀĲìĹ°ìĬ¤ëłĪ',
  'ĠëĶ°ëĿ¼',
  'ĠìĽĥê³ł',
  'ĠìŀĪëĬĶ',
  'Ġê±¸',
  'Ċ',
  'ëĦĪìĿĺ',
  'ĠíĸīëıĻìĹĲ',
  'ĠìĦ¤ëłĪìĸ´',
  'íķĺê³ł',
  'ĠëĴ¤ì²ĻìĿ´ëĭ¤ê°Ģ',
  'Ċ',
  'ì§ĢìĥĪ',
  'ìļ´',
  'Ġë°¤ìĿ´',
  'Ġë§İìķĦ',
  'ì§ĢëĬĶëį°',
  'Ċ',
  'ìĿ´ê±´',
  'ĠëĪĦê°Ģ',
  'Ġë´ĲëıĦ',
  'ĠìĤ¬ëŀĳìĿ¼',
  'ĠíħĲëį°',
  'Ċ',
  'ì¢ħìĿ¼',
  'Ġíķ¨ê»ĺë©´',
  'Ġì§Īë¦´',
  'ĠíħĲëį°',
  'Ċ',
  'ëĤĺ',
  'ĠëıĮìķĦìĦľëıĦ',
  'Ġìĺ¨íĨµ',
  'ĠëĦĪìĿ¸',
  'Ġê±´',
  'Ċ',
  'ìķĦë¬´ëŀĺëıĦ',
  'ĠìĤ¬ëŀĳìĿ¸ê°Ģ',
  'Ġë´Ĳ',
  'Ċ',
  'ìłĲìłĲ',
  'ĠëĦĪìĻĢ',
  'Ġíķĺê³ł',
  'Ġìĭ¶ìĿĢ',
  'ĠìĿ¼ëĵ¤',
  'ĠìĥĿê°ģíķĺë©´ìĦľ',
  'Ċ',
  'íķĺë£°',
  'Ġë³´ëĤ¸',
  'ĠëĤłìĿ´',
  'Ġë§İìķĦ',
  'ì§ĢëĬĶëį°',
  'Ċ',
  'ìĿ´ê±´',
  'ĠëĪĦê°Ģ',
  'Ġë´ĲëıĦ',
  'ĠìĤ¬ëŀĳìĿ¼',
  'ĠíħĲëį°',
  'Ċ',
  'ì¢ħìĿ¼',
 

이번 배치의 크기가 3이라고 가정하고 이번 배치의 입력값을 만들어 보겠습니다.

In [73]:
sentences = [df.lyric[0],df.lyric[1],df.lyric[2]]
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length",
    max_length=200,
    truncation=True,
)

In [64]:
len(df.lyric[0]), len(df.lyric[1]), len(df.lyric[2])

(349, 482, 576)

`batch_inputs`의 내용을 확인해보겠습니다.

In [74]:
batch_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [75]:
batch_inputs['input_ids']

[[923,
  757,
  1423,
  1299,
  18865,
  199,
  94779,
  974,
  199,
  3812,
  324,
  20686,
  3264,
  199,
  745,
  10246,
  724,
  2688,
  1107,
  2400,
  199,
  13718,
  1089,
  2840,
  774,
  511,
  199,
  745,
  25665,
  13472,
  780,
  57918,
  199,
  51599,
  560,
  1955,
  2467,
  14603,
  199,
  2553,
  2533,
  1872,
  19919,
  4564,
  199,
  2770,
  6998,
  72786,
  4564,
  199,
  285,
  39478,
  4389,
  11739,
  777,
  199,
  13148,
  14831,
  663,
  199,
  2172,
  1014,
  1423,
  1299,
  9729,
  21489,
  199,
  32530,
  4724,
  3413,
  2467,
  14603,
  199,
  2553,
  2533,
  1872,
  19919,
  4564,
  199,
  2770,
  6998,
  72786,
  4564,
  199,
  285,
  39478,
  4389,
  11739,
  777,
  199,
  13148,
  14831,
  663,
  199,
  745,
  82693,
  2688,
  1107,
  2400,
  199,
  1134,
  2120,
  1089,
  22237,
  199,
  264,
  21344,
  21533,
  558,
  3338,
  199,
  1890,
  5752,
  5386,
  319,
  5774,
  199,
  1913,
  1495,
  974,
  13769,
  199,
  682,
  1225,
  1138,
  558,
  3338,


In [76]:
batch_inputs['attention_mask']

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],

In [63]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

NameError: ignored

# 학습데이터 구축
학습데이터를 만듭니다.

In [69]:
df.head()

,pages,rank,title,lyric,artist,genres,writer,lyric_mecab,word
0,1,1,사랑인가 봐,너와 함께 하고 싶은 일들을\n상상하는 게\n요즘 내 일상이 되고\n너의 즐거워하는...,멜로망스,"발라드, 국내드라마",김민석 (멜로망스),"[(너, NP), (와, JKB), (함께, MAG), (하, VV), (고, EC...","[너, 와, 함께, 하, 고, 싶, 은, 일, 들, 을, 상상, 하, 는, 게, 요..."
1,1,2,취중고백,뭐하고 있었니 늦었지만\n잠시 나올래\n너의 집 골목에 있는\n놀이터에 앉아 있어\...,김민석 (멜로망스),발라드,김희탐,"[(뭐, IC), (하, VV), (고, EC), (있, VX), (었, EP), ...","[뭐, 하, 고, 있, 었, 니, 늦, 었, 지만, 잠시, 나올래, 너, 의, 집,..."
2,1,3,듣고 싶을까,우리 함께 듣던 그 노랠 듣고 싶을까\n듣고 나서 잠시 날 생각은 할까\n아주 혹시...,MSG워너비(M.O.M),발라드,Dr.JO,"[(우리, NP), (함께, MAG), (듣, VV), (던, ETM), (그, M...","[우리, 함께, 듣, 던, 그, 노랠, 듣, 고, 싶, 을까, 듣, 고, 나, 서,..."
3,1,4,사랑은 늘 도망가,눈물이 난다 이 길을 걸으면\n그 사람 손길이 자꾸 생각이 난다\n붙잡지 못하고 가...,임영웅,"발라드, 국내드라마",강태규,"[(눈물, NNG), (이, JKS), (난다, VV+EF), (이, MM), (길...","[눈물, 이, 난다, 이, 길, 을, 걸으면, 그, 사람, 손길, 이, 자꾸, 생각..."
4,1,5,언제나 사랑해,둘이 손잡고 걸을 때\n마주 보며 또 웃을 때\n사랑한다 말하지 않아도 알아\n널 ...,케이시 (Kassy),발라드,조영수,"[(둘, NR), (이, JKS), (손잡, VV), (고, EC), (걸, VV)...","[둘, 이, 손잡, 고, 걸, 을, 때, 마주, 보, 며, 또, 웃, 을, 때, 사..."


In [45]:
from ratsnlp.nlpbook.generation import NsmcCorpus, GenerationDataset
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
corpus = NsmcCorpus()
train_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 10.073 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 부정 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: ▁부정 ▁아 ▁더 빙 .. ▁진짜 ▁짜 증 나 네 요 ▁목소리 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
INFO:ratsnlp:features: GenerationFeatures(input_ids=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], labels=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 4821

# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [13]:
val_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 2.561 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 긍정 굳 ㅋ
INFO:ratsnlp:tokens: ▁긍정 ▁굳 ▁ ᄏ </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
INFO:ratsnlp:features: GenerationFeatures(input_ids=[16420, 12969, 739, 605, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], labels=[16420, 12969, 739, 605, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

# 모델 초기화
프리트레인이 완료된 GPT2 모델을 읽고, 문장 생성 모델을 초기화합니다.

In [14]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    args.pretrained_model_name
)

# 학습 준비
Task와 Trainer를 준비합니다.

In [18]:
from ratsnlp.nlpbook.generation import GenerationTask
task = GenerationTask(model, args)

NameError: ignored

In [16]:
trainer = nlpbook.get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


# 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-generation`)에 저장됩니다.

In [17]:
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.656   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


In [59]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [60]:
from Korpora import Korpora
nsmc = Korpora.load('nsmc', force_download = True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 100MB/s]                             
[nsmc] download ratings_test.txt: 4.90MB [00:00, 44.2MB/s]                            


In [61]:
import os
def write_lines(path, lines):
    with open(path, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(f'{line}\n')

write_lines("/content/train.txt", nsmc.train.get_all_texts())
write_lines("/content/test.txt", nsmc.test.get_all_texts())

In [64]:
# GPT 토크나이저 구출
import os
os.makedirs('/Mydrive/nlpbook/bbpe', exist_ok=True)

In [66]:
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],
    vocab_size=10000,
    special_tokens=["[PAD]"]
)
bytebpe_tokenizer.save_model("/MyDrive/nlpbook/bbpe")

Exception: ignored

In [55]:
ballad.lyric

0        너와 함께 하고 싶은 일들을\n상상하는 게\n요즘 내 일상이 되고\n너의 즐거워하는...
1        뭐하고 있었니 늦었지만\n잠시 나올래\n너의 집 골목에 있는\n놀이터에 앉아 있어\...
2        우리 함께 듣던 그 노랠 듣고 싶을까\n듣고 나서 잠시 날 생각은 할까\n아주 혹시...
3        눈물이 난다 이 길을 걸으면\n그 사람 손길이 자꾸 생각이 난다\n붙잡지 못하고 가...
4        둘이 손잡고 걸을 때\n마주 보며 또 웃을 때\n사랑한다 말하지 않아도 알아\n널 ...
                               ...                        
71036    가슴이 미어져 혼자서 괴로울 때\n나를 기억해줘\n지치고 지쳐서\n기댈 힘조차 없게...
71037    매일 널 찾게 돼\n뻔하지 않아 우리 둘은\n내 옆에 널 보면\n자꾸 사랑에 빠질듯...
71038    시간이 가면 모두 괜찮을거야\n그 말을 믿은 내가 참 바보같았어\n또 하루 지나가도...
71039    바람에 실려 오는 그리움이\n귓가에 남아있는 너의 목소리\n오늘만 살아가는 마른 입...
71040    마지막이 그려지던 우리\n그때 넌 어땠었을까\n많이 미안하다고\n하고 돌아선 너는\...
Name: lyric, Length: 71041, dtype: object

In [51]:
ballad.lyric.to_csv('/gdrive/My drive/nlpbook/bbpe/train.txt')

In [52]:
pd.read_csv('/gdrive/My drive/nlpbook/bbpe/train.txt')

,Unnamed: 0,lyric
0,0,너와 함께 하고 싶은 일들을\n상상하는 게\n요즘 내 일상이 되고\n너의 즐거워하는...
1,1,뭐하고 있었니 늦었지만\n잠시 나올래\n너의 집 골목에 있는\n놀이터에 앉아 있어\...
2,2,우리 함께 듣던 그 노랠 듣고 싶을까\n듣고 나서 잠시 날 생각은 할까\n아주 혹시...
3,3,눈물이 난다 이 길을 걸으면\n그 사람 손길이 자꾸 생각이 난다\n붙잡지 못하고 가...
4,4,둘이 손잡고 걸을 때\n마주 보며 또 웃을 때\n사랑한다 말하지 않아도 알아\n널 ...
...,...,...
71036,71036,가슴이 미어져 혼자서 괴로울 때\n나를 기억해줘\n지치고 지쳐서\n기댈 힘조차 없게...
71037,71037,매일 널 찾게 돼\n뻔하지 않아 우리 둘은\n내 옆에 널 보면\n자꾸 사랑에 빠질듯...
71038,71038,시간이 가면 모두 괜찮을거야\n그 말을 믿은 내가 참 바보같았어\n또 하루 지나가도...
71039,71039,바람에 실려 오는 그리움이\n귓가에 남아있는 너의 목소리\n오늘만 살아가는 마른 입...


In [48]:
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=['/content/train.txt', '/content/test.txt'],
    vocab_size = 10000,
    special_tokens = ["[PAD]"]
)
bytebpe_tokenizer.save_model('/gdrive/My drive/nlpbook/bbpe')

Exception: ignored



---



# 프리트레인 마친 모델로 문장생성

# 모델 로딩
프리트레인한 GPT2 모델과 토크나이저를 읽어 들입니다.

In [19]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)
model.eval() # 학습 때만 필요한 기능들을 꺼서 평가모드로 동작

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [21]:
model.generate(df['lyric'])

AttributeError: ignored

In [20]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


## 프롬프트 준비

언어모델에 넣을 프롬프트를 준비합니다.

In [46]:
data1 = "자세히 보아야\n 예쁘다\n 오래 보아야\n 사랑스럽다\n 너도\n 그렇다"

In [47]:
input_ids = tokenizer.encode(data1 , return_tensors="pt")
input_ids

tensor([[14328, 25601,   375,  9182,  7748,  9503, 10714, 25601,   375, 10270,
         12207,  9503, 10099, 30286, 11928,  7182]])

## Greedy Search


다음 단어 확률 분포에서 최대 확률을 내는 단어들을 리턴합니다. 
여러 번 수행하더라도 생성 결과가 바뀌지 않습니다 (`do_sample=False`).
`max_length`는 생성 최대 길이이며 이보다 길거나, 짧더라도 EOD 토큰 등 스페셜 토큰이 나타나면 생성을 중단합니다. `min_length`는 생성 최소 길이이며 이보다 짧은 구간에서 EOD 등 스페셜 토큰이 등장해 생성이 중단될 경우 해당 토큰이 나올 확률을 0으로 수정하여 문장 생성이 종료되지 않도록 강제합니다.

In [53]:
import torch
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=5,
        max_length=30,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

자세히 보아야
 예쁘다
 오래 보아야
 사랑스럽다
 너도
 그렇다!
이제부터
이제부터
이제부터
이


In [27]:
df['lyric'][0]

'너와 함께 하고 싶은 일들을\n상상하는 게\n요즘 내 일상이 되고\n너의 즐거워하는 모습을 보고 있으면\n자연스레 따라 웃고 있는 걸\n너의 행동에 설레어하고 뒤척이다가\n지새운 밤이 많아지는데\n이건 누가 봐도 사랑일 텐데\n종일 함께면 질릴 텐데\n나 돌아서도 온통 너인 건\n아무래도 사랑인가 봐\n점점 너와 하고 싶은 일들 생각하면서\n하룰 보낸 날이 많아지는데\n이건 누가 봐도 사랑일 텐데\n종일 함께면 질릴 텐데\n나 돌아서도 온통 너인 건\n아무래도 사랑인가 봐\n너의 행복해하는 모습을 보고 있으면\n나도 모르게 따라 웃는데\n이 정도면 알아줄 만하잖아\n너도 용기 낼만 하잖아\n나만 이런 게 아니라면\n우리 만나볼 만하잖아\n아무래도 사랑인가 봐'

## Beam Search 

Beam Search는 다음 단어 확률 분포에서 `num_beams`만큼의 경우의 수를 남겨가면서 문장을 생성합니다. Beam search는 Greedy search보다 계산량이 많지만 좀 더 확률값이 높은 문장을 생성할 수 있습니다.

In [54]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=30,
        num_beams=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

자세히 보아야
 예쁘다
 오래 보아야
 사랑스럽다
 너도
 그렇다네
예쁘다네
예쁘다네
예쁘


`num_beams=1`로 설정하면 정확히 Greedy search와 동일하게 작동합니다.


In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        num_beams=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

## 반복 줄이기

### 반복되는 n-gram 사이즈를 지정하기

위의 예시를 보면 `"그럼, 그건 뭐예요?"`이 반복됩니다. 이를 아래와 같이 지정해 반복을 방지합니다. 3개 이상의 토큰이 반복될 경우 해당 3-gram 등장 확률을 0으로 만들어 생성 결과에서 배제합니다.


In [57]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        no_repeat_ngram_size=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

자세히 보아야
 예쁘다
 오래 보아야
 사랑스럽다
 너도
 그렇다!
이제부터
이제부터는
이렇게
이런저런 이야기를
이젠
이제는
이것저것
이것도 다
이


### repetition penalty

repetition penalty로 반복을 통제할 수도 있습니다. 다음과 같이 실행하면 되며 그 범위는 1 이상의 값을 가져야 합니다. 1이라면 아무런 패널티를 적용하지 않는게 됩니다.


In [58]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

자세히 보아야
 예쁘다
 오래 보아야
 사랑스럽다
 너도
 그렇다!
이제부터
이제부터
이제부터
이제부터
이제부터
이제부터
이제부터
이제부터
이


`repetition_penalty` 값이 클 수록 패널티가 세게 적용됩니다.

In [60]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=150,
        repetition_penalty=1.1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

자세히 보아야
 예쁘다
 오래 보아야
 사랑스럽다
 너도
 그렇다!
이제부터라도
사랑스러운 내 모습을
보고 싶다.
그런데
아무래도 내가
나에게
사랑을 고백하는 것 같아서
어쩌면 이럴지도 모르겠다.
하지만
그렇게만 된다면
내게
사랑은 더 이상
없어져야 할 것 같다.
그래서 나는
사랑을 고백하고 싶어한다.
나는
사랑을 고백하고 싶어한다.
사랑은 더 이상
없어져야 할 것 같다.
사랑은 더 이상
없어져야 할 것 같다.
사랑은 더 이상
없어져야 할 것 같다.
사랑은 더 이상
없어져야 할 것 같다.
사랑은 더 이상
없어져야 할


In [61]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.2,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

자세히 보아야
 예쁘다
 오래 보아야
 사랑스럽다
 너도
 그렇다!
이제부터라도
사랑스러운 내 모습을 보여줘야겠어요
#셀카 #selfie #instagood #o


In [62]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

자세히 보아야
 예쁘다
 오래 보아야
 사랑스럽다
 너도
 그렇다!
이제부터라도 내 마음을 다잡아서
내 마음속에 있는 모든 것을 알아보고자
사랑스러운 나의 모습을 찾아보자!</d> #20180


## top-k sampling

지금까지는 생성을 반복하더라도 그 결과가 동일한 샘플링 방식을 살펴봤습니다. top-k sampling은 다음 단어를 뽑을 때 확률값 기준 가장 큰 k개 가운데 하나를 선택하는 기법입니다. 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, k개 안에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있습니다. 따라서 top-k sampling은 매 시행 때마다 생성 결과가 달라집니다. k는 1 이상의 값을 지녀야 합니다.

In [63]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

자세히 보아야
 예쁘다
 오래 보아야
 사랑스럽다
 너도
 그렇다구
난 그대는
그대는
네 앞이
예쁘다는 이유만으로
어디로 치면? 라고 말하겠지
예쁘다는 이유


k=1일 경우 Greedy search와 동일합니다. 

In [64]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

자세히 보아야
 예쁘다
 오래 보아야
 사랑스럽다
 너도
 그렇다!
이제부터
이제부터
이제부터
이제부터
이제부터
이제부터
이제부터
이제부터
이


## top-k sampling + temperature scaling

top-k sampling은 temperature scaling과 동시에 적용할 수 있습니다. 그 값에 따라 다음과 같은 효과가 납니다.

(1) t가 0에 가까워질 수록 토큰 분포가 sharp해진다 > 1등 토큰이 뽑힐 확률이 그만큼 높아진다 > do_sample=True이지만 사실상 greedy decoding이 된다

In [65]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

자세히 보아야
 예쁘다
 오래 보아야
 사랑스럽다
 너도
 그렇다!
이제부터
이제부터
이제부터
이제부터
이제부터
이제부터
이제부터
이제부터
이


(2) t=1이라면 모델 출력 분포를 그대로 사용한다 > 하지만 샘플링 방식을 사용하기 때문에 생성할 때마다 다른 문장이 나온다

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

(3) t를 키울수록 토큰 분포가 uniform해진다 > 사실상 uniform sampling이 된다, 생성 품질이 악화할 가능성이 높아진다

In [42]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=100000000.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

Input length of input_ids is 192, but ``max_length`` is set to 50.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


너와 함께 하고 싶은 일들을
상상하는 게
요즘 내 일상이 되고
너의 즐거워하는 모습을 보고 있으면
자연스레 따라 웃고 있는 걸
너의 행동에 설레어하고 뒤척이다가
지새운 밤이 많아지는데
이건 누가 봐도 사랑일 텐데
종일 함께면 질릴 텐데
나 돌아서도 온통 너인 건
아무래도 사랑인가 봐
점점 너와 하고 싶은 일들 생각하면서
하룰 보낸 날이 많아지는데
이건 누가 봐도 사랑일 텐데
종일 함께면 질릴 텐데
나 돌아서도 온통 너인 건
아무래도 사랑인가 봐
너의 행복해하는 모습을 보고 있으면
나도 모르게 따라 웃는데
이 정도면 알아줄 만하잖아
너도 용기 낼만 하잖아
나만 이런 게 아니라면
우리 만나볼 만하잖아
아무래도 사랑인가 봐"



## top-p sampling

top-p sampling은 다음 단어를 뽑을 때 누적 확률값이 p 이하인 단어들 가운데 하나를 선택하는 기법입니다. 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, 누적 확률값 p 이하에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있습니다. 따라서 top-p sampling은 매 시행 때마다 생성 결과가 달라집니다. p는 확률이기 때문에 0~1 사이의 값을 지녀야 합니다. p가 1이라면 어휘 집합에 있는 모든 단어를 대상으로 샘플링하기 때문에 top-p sampling 효과가 사라집니다. p가 1보다 약간 작다면 확률값이 낮은 일부 단어들을 다음 단어 후보에서 제거해 생성 품질을 높입니다. 

In [43]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

Input length of input_ids is 192, but ``max_length`` is set to 50.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


너와 함께 하고 싶은 일들을
상상하는 게
요즘 내 일상이 되고
너의 즐거워하는 모습을 보고 있으면
자연스레 따라 웃고 있는 걸
너의 행동에 설레어하고 뒤척이다가
지새운 밤이 많아지는데
이건 누가 봐도 사랑일 텐데
종일 함께면 질릴 텐데
나 돌아서도 온통 너인 건
아무래도 사랑인가 봐
점점 너와 하고 싶은 일들 생각하면서
하룰 보낸 날이 많아지는데
이건 누가 봐도 사랑일 텐데
종일 함께면 질릴 텐데
나 돌아서도 온통 너인 건
아무래도 사랑인가 봐
너의 행복해하는 모습을 보고 있으면
나도 모르게 따라 웃는데
이 정도면 알아줄 만하잖아
너도 용기 낼만 하잖아
나만 이런 게 아니라면
우리 만나볼 만하잖아
아무래도 사랑인가 봐



p가 0에 가까울 경우 Greedy search와 비슷해 집니다.

In [44]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

Input length of input_ids is 192, but ``max_length`` is set to 50.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


너와 함께 하고 싶은 일들을
상상하는 게
요즘 내 일상이 되고
너의 즐거워하는 모습을 보고 있으면
자연스레 따라 웃고 있는 걸
너의 행동에 설레어하고 뒤척이다가
지새운 밤이 많아지는데
이건 누가 봐도 사랑일 텐데
종일 함께면 질릴 텐데
나 돌아서도 온통 너인 건
아무래도 사랑인가 봐
점점 너와 하고 싶은 일들 생각하면서
하룰 보낸 날이 많아지는데
이건 누가 봐도 사랑일 텐데
종일 함께면 질릴 텐데
나 돌아서도 온통 너인 건
아무래도 사랑인가 봐
너의 행복해하는 모습을 보고 있으면
나도 모르게 따라 웃는데
이 정도면 알아줄 만하잖아
너도 용기 낼만 하잖아
나만 이런 게 아니라면
우리 만나볼 만하잖아
아무래도 사랑인가 봐



## 통합 적용

저희가 실습에 사용하고 있는 허깅페이스(huggingface) 라이브러리의 구현상 적용 순서는 다음과 같습니다.

- _get_logits_processor
  - RepetitionPenalty
  - NoRepeatNGramLogits
  - MinLengthLogits
- _get_logits_warper
  - TemperatureLogits
  - TopKLogits
  - TopPLogits

유효한 설정들을 종합 적용해 문장을 생성하는 코드는 다음과 같습니다. 샘플링(top-k, top-p)을 적용하기 때문에 시행 때마다 다른 문장이 생성됩니다.

In [56]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3,
        temperature=0.9,
        top_k=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

자세히 보아야
 예쁘다
 오래 보아야
 사랑스럽다
 너도
 그렇다 해도 내겐 그 사람 이름이 중요하지 않을까?
아니면 내가 얼마나 많이 알지?
가까운 시일 안에 나를 알아보는 것이 나의 가장 중요한 일이다.</d> 이번


https://github.com/gyunggyung/KoGPT2-FineTuning/blob/master/Colab.ipynb

In [72]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import logging

#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#my_drive = GoogleDrive(gauth)

In [73]:
!pip install -r drive/'My Drive'/'KoGPT2-FineTuning_pre'/requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'drive/My Drive/KoGPT2-FineTuning_pre/requirements.txt'
